# Consolidating the ARIES datasets:

## Background

Excerpt from the IndyCrash [project wiki](https://osf.io/hr7kz/wiki/home/):

>Hi, my name's Dan, I'm a resident pediatrician in Indianapolis. Several years ago, my good friend Mike was bicycling through the streets of Philadelphia when a car T-boned him after it ran a red light. Mike shattered the tip of his right elbow and his head struck straight into the windshield. He likely would not have survived if he had not been wearing a helmet. He spent months recovering from complex orthopedic surgery with physical rehab.
>
>I love bicycling/walking (aka active mobility). It’s a great way to build exercise into the daily routine while reducing your carbon footprint. But Mike's experience underscores why many people don't engage in active transportation--they don't feel safe on our streets. I want that to change.
>
>This project's goal is to rebuild the environment of streets in Indy. We will engage community members around the city to design interventions for the highest risk regions to pedestrians/bicyclists. Then we'll use randomized controlled trials to test what helps protect people with the best efficacy and dollar efficiency.

## Intro

This preliminary notebook will demonstrate a method for building the datasets to be used for this project. The data is derived from files made publically available from the ARIES database maintained by [Indiana State Police](https://hub.mph.in.gov/dataset/aries-crash-data-2007-2017):

>The Automated Reporting Information Exchange System (ARIES) is the State of Indiana’s crash repository. Crash data is generated through first responder crash reports and collected within ARIES. Data is available for 2007 to [2019]. Examples of this data include crash details such as, vehicle information, road conditions, crash severity, weather conditions, location, date, and time.

Sidenote: If you are familiar with the process of data ingestion, you may be wondering why I am not using the API on the ARIES website to run more targetted queries rather than downloading each individual file. Unfortunately, the API only reliably works for half of the datasets without returning an error, so I decided it was more important to have complete data rather than a more efficient process.

## Part 1: Load and combine the data

Let's start by prepping the needed R libraries.

In [1]:
library(readr)
library(dplyr)
library(feather)
library(tibble)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



I started by looping over every dataset, collecting the variables I want, and then adding them to a master table.

In [4]:
#Use a for loop to consolidate the ARIES datasets from 2007-Present.

tbl <- tibble()

for (year in 2007:2019) {
    dataset <- paste0("raw_aries_data/aries_crash_data_",year,".csv")
    locTbl <- read_csv(dataset,
                       #col_types argument is manually setting the types for each variable. This is because the sheer volume of missing data made it hard for the parser infer types itself.
                       col_types="dddcddcccdcdcdccdcdcdcdcccccddcdccccdtcdddddccccdcddccccccdcdccdcdcdcccccccccccdddccccdcccccdcdcccccdcdccdc") %>% 
                #Variables of interest to this project included identifiers, injury characterization, geographic position, time, and characterization of the physical environment
                #See ARIES data dictionary for all the variables. https://hub.mph.in.gov/dataset/aries-crash-data-2007-2017/resource/f61a5dcb-5ca3-485a-9ecf-cd3d8740dc9b?inner_span=True
                select(INDIVIDUAL_MR_RECORD,
                            LATDECIMALNMB,
                            LONGDECIMALNMB,
                            COUNTYDESCR,
                            UNIQUELOCATIONID,
                            COLLISION_YEAR,
                            COLLDTE,
                            SPEEDLIMITTXT,
                            TRAFFICCNTRLDESCR,
                            TRAFFICCNTLOPIND,
                            RUMBLESTRIPIND,
                            SURFACETYPEDESCR,
                            ROADTYPEDESCR,
                            SCHOOLZONEIND,
                            MANNERCOLLDESCR,
                            COLLEVENTDESCR,
                            INJUREDNMB,
                            DEADNMB, 
                            UNITNMB,
                            OCCUPSNMB,
                            PERSONNMB,             
                            PERSONTYPEDESCR,
                            AGE_GRP)
    tbl <- bind_rows(tbl,locTbl)
}

str(tbl)

Warning message:
“7712 parsing failures.
  row              col expected actual                                       file
41379 SURFACETYPECDE   a double      + 'raw_aries_data/aries_crash_data_2007.csv'
62021 WEATHERCDE       a double      + 'raw_aries_data/aries_crash_data_2007.csv'
62021 PRIMARYFACTORCDE a double      + 'raw_aries_data/aries_crash_data_2007.csv'
62021 UNITTYPECDE      a double      + 'raw_aries_data/aries_crash_data_2007.csv'
71273 UNITTYPECDE      a double      + 'raw_aries_data/aries_crash_data_2007.csv'
..... ................ ........ ...... ..........................................
See problems(...) for more details.
”Warning message:
“472 parsing failures.
  row              col           expected      actual                                       file
21431 VEHMODELTXT      delimiter or quote I           'raw_aries_data/aries_crash_data_2008.csv'
21431 OCCUPSNMB        a double           INDIANA     'raw_aries_data/aries_crash_data_2008.csv'
21431 ROADTYPECDE

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	7244543 obs. of  23 variables:
 $ INDIVIDUAL_MR_RECORD: num  9.01e+08 9.01e+08 9.01e+08 9.01e+08 9.01e+08 ...
 $ LATDECIMALNMB       : num  0 40.7 39.2 40.2 41.6 ...
 $ LONGDECIMALNMB      : num  0 -85.4 -85.9 -85.4 -86.2 ...
 $ COUNTYDESCR         : chr  "Bartholomew" "Huntington" "Bartholomew" "Delaware" ...
 $ UNIQUELOCATIONID    : chr  "COLUMBUSSHOPPINGCENTER" "HUNTINGTONAVE" "E25THST" "KILGOREAVE" ...
 $ COLLISION_YEAR      : num  2007 2007 2007 2007 2007 ...
 $ COLLDTE             : chr  "2007.06.22" "2007.11.13" "2007.06.09" "2007.02.27" ...
 $ SPEEDLIMITTXT       : chr  "15" "0" "15" "20" ...
 $ TRAFFICCNTRLDESCR   : chr  "None" "None" "None" "None" ...
 $ TRAFFICCNTLOPIND    : chr  NA NA NA NA ...
 $ RUMBLESTRIPIND      : chr  "N" "N" "N" "N" ...
 $ SURFACETYPEDESCR    : chr  "ASPHALT" "ASPHALT" "ASPHALT" "ASPHALT" ...
 $ ROADTYPEDESCR       : chr  "Private Drive" "Private Drive" "Private Drive" "Private Drive" ...
 $ SCHOOLZONEIND    

We end up with 7 million observations of people involved in crashes around the whole state of Indiana from 2007-2019. At regular stages, I will be storing these intermediate datasets in "feather" files. Feather files are nice because they were designed to be compatible with all the major datascience languages such as R, Python, and Julia.

In [6]:
# Fast, language-agnostic format for tabular data storage on disk

write_feather(tbl, "intermediate_data_sets/raw_pasted_ARIES.feather")

## Part 2: Quality measures

Next we'll make some changes that should apply to all analyses moving forward for quality purposes.

In [7]:
tbl <- tbl %>% 
    # Delete observations without valid latitude or longitude
    filter(LATDECIMALNMB != 0 & LONGDECIMALNMB != 0) %>%
    # Delete duplicate rows
    distinct() %>%
    # Give a unique identifier to each observational unit at the level of individual person
    mutate(personKey = 1:n())
            
str(tbl)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	6418818 obs. of  24 variables:
 $ INDIVIDUAL_MR_RECORD: num  9.01e+08 9.01e+08 9.01e+08 9.01e+08 9.01e+08 ...
 $ LATDECIMALNMB       : num  40.7 39.2 40.2 41.6 40 ...
 $ LONGDECIMALNMB      : num  -85.4 -85.9 -85.4 -86.2 -85.9 ...
 $ COUNTYDESCR         : chr  "Huntington" "Bartholomew" "Delaware" "St Joseph" ...
 $ UNIQUELOCATIONID    : chr  "HUNTINGTONAVE" "E25THST" "KILGOREAVE" "WIRELANDRD" ...
 $ COLLISION_YEAR      : num  2007 2007 2007 2007 2007 ...
 $ COLLDTE             : chr  "2007.11.13" "2007.06.09" "2007.02.27" "2007.12.30" ...
 $ SPEEDLIMITTXT       : chr  "0" "15" "20" "10" ...
 $ TRAFFICCNTRLDESCR   : chr  "None" "None" "None" "None" ...
 $ TRAFFICCNTLOPIND    : chr  NA NA NA NA ...
 $ RUMBLESTRIPIND      : chr  "N" "N" "N" "N" ...
 $ SURFACETYPEDESCR    : chr  "ASPHALT" "ASPHALT" "ASPHALT" "ASPHALT" ...
 $ ROADTYPEDESCR       : chr  "Private Drive" "Private Drive" "Private Drive" "Private Drive" ...
 $ SCHOOLZONEIND       : chr 

In [8]:
write_feather(tbl, "intermediate_data_sets/pasted_ARIES_and_personIDs.feather")

In [2]:
tbl <- read_feather("intermediate_data_sets/pasted_ARIES_and_personIDs.feather")

## Part 3: Add useful variables

Throughout this project we will need an easy way to know if each observation was involved in an "active crash" that involved modes of active mobility like walking or bicycling.

You will see that the preexisting "person type description" marks each observation of a person by how they were getting around. Two of the options include "Pedal Cyclist" or "Pedestrian". Additionally, the "collision event description" variable also sometimes says that a "Pedestrian" or "Bicyclist" was hit. Our new variables will combine both of these sources of information.

It's important to note here that right now we are marking at the observational level of each individual person-- aka whether each individual was either themself using active mobility or they directly hit somebody using active mobility. I am explicitly clarifying this, because below we will be making a separate table that stores data at the observational level of each crash, which will try and encompass what we know about every person involved in that particular crash.

Hopefully, this will make more sense as we work through things.

In [8]:
# Show how to identify collisions involving active mobility

print(tbl %>% group_by(PERSONTYPEDESCR) %>% summarize(count = n()))
tbl %>% group_by(COLLEVENTDESCR) %>% summarize(count = n())

# A tibble: 9 x 2
  PERSONTYPEDESCR                 count
  <chr>                           <int>
1 Animal Drawn Vehicle Operator     280
2 Driver                        3573573
3 Injured                        152719
4 Other                             140
5 Owner Trailer                   80886
6 Owner Vehicle                 2440048
7 Pedal Cyclist                   11766
8 Pedestrian                      21421
9 NA                             137889


COLLEVENTDESCR,count
<chr>,<int>
05,1
06,1
11,1
12,3
Animal Drawn Vehicle,10939
Animal Other Than Deer,23234
Another Motor Vehicle,5064198
Bicycle,17755
Bridge Overhead Structure,3976


In [9]:
# Function to consolidate variables based on mode of active mobility

assignPersonMode = function(x, y) {
    if ((x %in% "Pedestrian" & y %in% "Bicycle") | (x %in% "Pedal Cyclist" & y %in% "Pedestrian")) {
        return("Both")
    }
    else if (any(x %in% "Pedestrian", y %in% "Pedestrian")) {
        return("Pedestrian")
    }
    else if (any(x %in% "Pedal Cyclist", y %in% "Bicycle")) {
        return("Bicyclist")
    }
    else {
        return("Neither")
    }
}

In [10]:
# Make variable that categorizes each observation as involving either walking, bicycling, both, or neither

tbl <- tbl %>% rowwise() %>% mutate(personActiveMode = assignPersonMode(PERSONTYPEDESCR,COLLEVENTDESCR))

str(tbl)

Classes ‘rowwise_df’, ‘tbl_df’, ‘tbl’ and 'data.frame':	6418818 obs. of  25 variables:
 $ INDIVIDUAL_MR_RECORD: num  9.01e+08 9.01e+08 9.01e+08 9.01e+08 9.01e+08 ...
 $ LATDECIMALNMB       : num  40.7 39.2 40.2 41.6 40 ...
 $ LONGDECIMALNMB      : num  -85.4 -85.9 -85.4 -86.2 -85.9 ...
 $ COUNTYDESCR         : chr  "Huntington" "Bartholomew" "Delaware" "St Joseph" ...
 $ UNIQUELOCATIONID    : chr  "HUNTINGTONAVE" "E25THST" "KILGOREAVE" "WIRELANDRD" ...
 $ COLLISION_YEAR      : num  2007 2007 2007 2007 2007 ...
 $ COLLDTE             : chr  "2007.11.13" "2007.06.09" "2007.02.27" "2007.12.30" ...
 $ SPEEDLIMITTXT       : chr  "0" "15" "20" "10" ...
 $ TRAFFICCNTRLDESCR   : chr  "None" "None" "None" "None" ...
 $ TRAFFICCNTLOPIND    : chr  NA NA NA NA ...
 $ RUMBLESTRIPIND      : chr  "N" "N" "N" "N" ...
 $ SURFACETYPEDESCR    : chr  "ASPHALT" "ASPHALT" "ASPHALT" "ASPHALT" ...
 $ ROADTYPEDESCR       : chr  "Private Drive" "Private Drive" "Private Drive" "Private Drive" ...
 $ SCHOOLZONEIN

In [11]:
write_feather(tbl, "intermediate_data_sets/pasted_ARIES_and_IDs_mode.feather")

In [ ]:
# R keeps crashing when I don't reload the dataset at this point and try to go immediately to the next cell ¯\_(ツ)_/¯. For the interest of sanity, I'm just reloading the feather file I just saved.

tbl <- read_feather("intermediate_data_sets/pasted_ARIES_and_IDs_mode.feather")

str(tbl)

In [13]:
# Make mode of activity into a binary variable

tbl <- tbl %>% mutate(personActive = ifelse(personActiveMode %in% c("Bicyclist","Pedestrian","Both"),TRUE,FALSE))

str(tbl)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	6418818 obs. of  26 variables:
 $ INDIVIDUAL_MR_RECORD: num  9.01e+08 9.01e+08 9.01e+08 9.01e+08 9.01e+08 ...
 $ LATDECIMALNMB       : num  40.7 39.2 40.2 41.6 40 ...
 $ LONGDECIMALNMB      : num  -85.4 -85.9 -85.4 -86.2 -85.9 ...
 $ COUNTYDESCR         : chr  "Huntington" "Bartholomew" "Delaware" "St Joseph" ...
 $ UNIQUELOCATIONID    : chr  "HUNTINGTONAVE" "E25THST" "KILGOREAVE" "WIRELANDRD" ...
 $ COLLISION_YEAR      : num  2007 2007 2007 2007 2007 ...
 $ COLLDTE             : chr  "2007.11.13" "2007.06.09" "2007.02.27" "2007.12.30" ...
 $ SPEEDLIMITTXT       : chr  "0" "15" "20" "10" ...
 $ TRAFFICCNTRLDESCR   : chr  "None" "None" "None" "None" ...
 $ TRAFFICCNTLOPIND    : chr  NA NA NA NA ...
 $ RUMBLESTRIPIND      : chr  "N" "N" "N" "N" ...
 $ SURFACETYPEDESCR    : chr  "ASPHALT" "ASPHALT" "ASPHALT" "ASPHALT" ...
 $ ROADTYPEDESCR       : chr  "Private Drive" "Private Drive" "Private Drive" "Private Drive" ...
 $ SCHOOLZONEIND       : chr 

In [15]:
write_feather(tbl, "intermediate_data_sets/pasted_ARIES_and_IDs_mode_personActive.feather")

In [20]:
tbl <- read_feather("intermediate_data_sets/pasted_ARIES_and_IDs_mode_personActive.feather")

## Part 4: Derive a "tidy" table for crashes as the individual unit

One of the principles of ["tidy" data](https://tidyr.tidyverse.org/) is that observational units at different levels (aka at the level of each person versus each crash) should be stored in separate tables. Before knowing this, I learned firsthand how not following this principle can lead to inconsistent results from analyses.

In [22]:
crashTbl <- tbl %>% 
    # These variables are not applicable at the crash level of observational unit
    select(-COLLEVENTDESCR, -UNITNMB, -OCCUPSNMB, -PERSONNMB, -PERSONTYPEDESCR) %>%
    # 447589 has inconsistent values with its group
    filter(!personKey %in% 447589) %>%
    # Group observations by crash event ID
    group_by(INDIVIDUAL_MR_RECORD) %>%
    # Create summary variables for each crash event
    summarize(county = first(COUNTYDESCR),
              locationID = first(UNIQUELOCATIONID),
              long = first(LONGDECIMALNMB),
              lat = first(LATDECIMALNMB),
              year = first(COLLISION_YEAR),
              date = first(COLLDTE),
              childInvolved = any(AGE_GRP %in% c("<1 Year", "1-4 Years", "5-14 Years", "15-24 Years")),
              crashActive = any(personActive == TRUE),
              crashActiveMode = case_when(any(personActiveMode %in% "Both") ~ "Both",
                                          any(personActiveMode %in% "Pedestrian") & any(personActiveMode %in% "Bicyclist") ~ "Both",
                                          any(personActiveMode %in% "Pedestrian") ~ "Pedestrian",
                                          any(personActiveMode %in% "Bicyclist") ~ "Bicyclist",
                                          TRUE ~ "Neither"),
              roadType = first(ROADTYPEDESCR),
              speedLimit = mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE),
              trafficControl = first(TRAFFICCNTRLDESCR),
              workingTrafficCntl = first(TRAFFICCNTLOPIND),
              rumbleStrips = first(RUMBLESTRIPIND),
              surfaceType = first(SURFACETYPEDESCR),
              schoolZone = first(SCHOOLZONEIND),
              mannerOfCollision = first(MANNERCOLLDESCR),
              numberInjured = first(INJUREDNMB),
              numberDead = first(DEADNMB)
             ) %>%
    ungroup()

str(crashTbl)

# This consolidation method was validated by checking for distinct values in each variable with notes as below
    # These are still included in the dataset but note that crash 902089566 either took place on Industrial or International Dr;
    # 902319772 either took place on 2014-10-22 or 12; 
    # 903220875 either took place in 2018-08-15 or 09-15;
# There are many instances where speed limit recorded is inconsistent, we will average this value
# There are many instances where road type is inconsistent, I considered using the statistical mode, but the R function does not know how to handle ties. I will just select the first value in each group, this will be validated in road audits anyway down the line

Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = TRUE):
“NAs introduced by coercion”Warning message in mean(as.numeric(SPEEDLIMITTXT), na.rm = T

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	2316061 obs. of  20 variables:
 $ INDIVIDUAL_MR_RECORD: num  20087 89296 89299 102620 140414 ...
 $ county              : chr  "Marion" "Marion" "Marion" "Marion" ...
 $ locationID          : chr  "KENTUCKYAVE" "SHIGHSCHOOLRDWPOWELLRD" "MILHOUSERDRIVERSPORTCT" "1STAVECRAWFORDSVILLERD" ...
 $ long                : num  -86.3 -86.3 -86.3 -86.3 -86.3 ...
 $ lat                 : num  39.7 39.7 39.7 39.8 39.7 ...
 $ year                : num  2010 2007 2009 2007 2008 ...
 $ date                : chr  "2010.10.13" "2007.10.02" "2009.09.10" "2007.03.29" ...
 $ childInvolved       : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ crashActive         : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ crashActiveMode     : chr  "Neither" "Neither" "Neither" "Neither" ...
 $ roadType            : chr  "Private Drive" "Two Lanes (Two Way)" "Two Lanes (Two Way)" "Two Lanes (Two Way)" ...
 $ speedLimit          : num  15 25 25 30 40 15 25 25 30 45 ...
 $ tr

In [24]:
summary(crashTbl$childInvolved)

   Mode   FALSE    TRUE 
logical 1484070  831991 

In [27]:
summary(crashTbl$crashActive)

   Mode   FALSE    TRUE 
logical 2281862   34199 

In [25]:
write_feather(crashTbl, "ariesCrashes.feather")

## Postscript: Old cells used for validation, exploratory analysis, or troubleshooting

In [4]:
#This cell debugged the manual typing of variables

# probs <- tibble()

#ariesVars <- c("PERSONTYPEDESCR","GENDERCDE","AGE_GRP","INJSTATUSDESCR","INJNATUREDESCR","TESTGIVENDESCR","RESULTALCHTXT","RESULTDRUGIND", "COUNTYDESCR","COLLDTE","MOTORVEHINVOLVEDNMB","INJUREDNMB","DEADNMB","RDWYSUFFIXTXT","LATDECIMALNMB","LONGDECIMALNMB", "TRAFFICCNTLOPIND","AGGRESSIVEDRIVEIND","HITRUNIND","SCHOOLZONEIND","RUMBLESTRIPIND","CONSTRUCTIND","LIGHTCONDDESCR", "WEATHERDESCR","SURFACETYPECDE_CONDDESCR","TYPEDESCR","PRIMARYFACTORDESCR","MANNERCOLLDESCR","TRAFFICCNTRLDESCR", "UNITTYPEDESCR","OCCUPSNMB","SPEEDLIMITTXT","VEHUSEDESCR","ROADTYPEDESCR","TRAVDIRDESCR","EMGERENCY_RUN","PRECOLLACTDESCR")

# for (year in 2007:2019) {
#     dataset <- paste0("aries_crash_data_",year,".csv")
#     tbl <- read_csv(dataset,
#                        col_types="dddcddcccdcdcdccdcdcdcdcccccddcdccccdtcdddddccccdcddccccccdcdccdcdcdcccccccccccdddccccdcccccdcdcccccdcdccdc") %>% 
#                 select(7:9,18,20,23,25,26,31,34,40,42,43,45,51,52,53,54:58,60,62,63,65,67,69,79,83,87,91,94,96,98,99,104) %>%
#                 filter(PERSONTYPEDESCR %in% c("Pedal Cyclist","Pedestrian"))
#     locprobs <- problems(tbl) %>% 
#                 group_by(col,expected,actual) %>% 
#                 summarize(count = n()) %>%
#                 filter(col %in% ariesVars)
#     probs <- bind_rows(probs,locprobs)
# }

# probs

In [5]:
#This cell generated missing value percentages
# library(purrr)

# tbl %>% map(~ (mean(is.na(.))*100))

In [6]:
#This cell dropped variables that had missing values that were above 75% (Injury status and nature)

# tbl2 <- tbl %>% select(which(colMeans(is.na(.))*100 < 75))

# tbl2 %>% map(~ (mean(is.na(.))*100))

In [ ]:
# # This cell attempted to give each observation its own "unique" id by combining crash ID, vehicle ID, and person ID
# tbl <- tbl %>% mutate(nested_id = as.numeric(paste0(INDIVIDUAL_MR_RECORD,UNITNMB,PERSONNMB)))

# str(tbl)

# # We still have repeat nested_id's and not all of these repeats are "NA"
# paste0("Number of distinct index numbers: ",n_distinct(tbl$nested_id))
# paste0("Number of index numbers that are NA: ",sum(is.na(tbl$nested_id)))

# # This "toy" was used for validating the script below
# # toy <- tribble(
# #     ~a, ~b, ~nested_id,
# #     1, 2, 3, 
# #     1, 2, 3,
# #     1, 2, NA,
# #     1, 2, NA, 
# #     4, 5, 6
# # )

# # Identify cells that still have the same Nested_ID number
# duplicates <- tbl %>% group_by(nested_id) %>% filter(n()>1 & !(nested_id %in% NA)) %>% ungroup() %>% arrange(nested_id)

# # Each of these observations has a duplicate Nested_ID, but a slightly different combination of variable values. 
# # I'm saving these observations for reference later, but will keep the rows in the dataset.
# write_csv(duplicates,"duplicates.csv")

In [ ]:
# # This toy validated a function to assign personal mode of active mobility

# toyPersonMode <- tribble(
#     ~PERSONTYPEDESCR, ~COLLEVENTDESCR, ~UNIMPORTANT,
#     "Pedestrian", "Bicycle", 57,
#     "Pedal Cyclist", "Pedestrian", 92,
#     "Pedestrian", "Pedestrian",34,
#     "fsdsdf", "Pedestrian",34,
#     "dsfsd","Bicycle",54,
#     "Pedal Cyclist", "fsdfd",76,
#     "dfsdfd", "sdfsdfds",78,
#     NA,"sdasda",87,
#     NA, "Bicycle",98,
#     "fdsfsd", NA, 03,
#     "Pedestrian",NA,98    
# )

# toy <- toyPersonMode %>% rowwise() %>% mutate(personActiveMode = assignPersonMode(PERSONTYPEDESCR, COLLEVENTDESCR))

In [ ]:
# # This cell verified that there were crash ID's that were shared between distinct events

# # There are instances of duplicate crash ID's being used in distinct events, but they took place outside of the Indy metro region so I will not work to correct it in this analysis

# library(tidyr)

# testing <- tbl %>% 
#     select(-UNITNMB, -OCCUPSNMB, -PERSONNMB, -PERSONTYPEDESCR, -personKey, -personMode) %>%
#     unite(lat_long, LATDECIMALNMB, LONGDECIMALNMB) %>%
#     group_by(INDIVIDUAL_MR_RECORD) %>%
#     filter(n_distinct(lat_long)>1) %>%
#     ungroup() %>% 
#     arrange(INDIVIDUAL_MR_RECORD)

In [ ]:
# # This cell tried to catch variables whose associated crash indicated there was a pedestrian involved
# tbl <- tbl %>%
#     group_by(INDIVIDUAL_MR_RECORD) %>%
#     mutate(activeCrash = cumany(active %in% "active")) %>%
#     ungroup()

# str(tbl)

In [ ]:
# # This cell validated making the crashActiveMode variable

# toy <- tribble(
#     ~INDIVIDUAL_MR_RECORD, ~personActiveMode,
#     1, "Both",
#     1, "Neither",
#     2, "Pedestrian",
#     2, "Bicyclist",
#     3, "Pedestrian",
#     3, "Neither",
#     4, "Bicyclist",
#     4, "Neither",
#     5, "Neither",
#     5, "Neither"
# )

# toy %>% 
#     group_by(INDIVIDUAL_MR_RECORD) %>% 
#     summarize(crashActiveMode = case_when(any(personActiveMode %in% "Both") ~ "Both",
#                                           any(personActiveMode %in% "Pedestrian") & any(personActiveMode %in% "Bicyclist") ~ "Both",
#                                           any(personActiveMode %in% "Pedestrian") ~ "Pedestrian",
#                                           any(personActiveMode %in% "Bicyclist") ~ "Bicyclist",
#                                           TRUE ~ "Neither"))

In [ ]:
# # This cell validated making the crashActive variable

# toy <- tribble(
#     ~INDIVIDUAL_MR_RECORD, ~personActive,
#     1, TRUE,
#     1, TRUE,
#     2, TRUE,
#     2, FALSE,
#     3, FALSE,
#     3, FALSE
# )

# toy %>%
#     group_by(INDIVIDUAL_MR_RECORD) %>%
#     summarize(crashActive = any(personActive == TRUE))

In [ ]:
# # Make dataset of crashes that all involve active mobility

# activeTbl <- crashTbl %>% filter(activeCrash == TRUE)

# str(activeTbl)

In [ ]:
# # Make dataset of collisions that have no inclusion of active mobility

# inactiveTbl <- crashTbl %>% filter(activeCrash == FALSE)

# str(inactiveTbl)

In [ ]:
# # Verify there is no overlap in observations between the two tables

# overlap <- inner_join(activeTbl, inactiveTbl, by="INDIVIDUAL_MR_RECORD")

# str(overlap)